## Method3 (Basically, Method2 but without bbox)

In [1]:
import trimesh
import pyvista as pv
from utils import *

#### Load a file

In [2]:
mesh = pymesh.load_mesh("data/three_spheres.ply")

In [3]:
self_intersection_stats(mesh)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |    1608 |
+-----------------------------------+---------+
| Number of faces                   |    3204 |
+-----------------------------------+---------+
| Number of intersecting face pairs |     292 |
+-----------------------------------+---------+


In [4]:
visualize_intersection(mesh)

/home/kmaeda/anaconda3/envs/pymesh_env/lib/python3.10/site-packages/pyvista/core/pointset.py:1365: PyVistaDeprecationWarning: The current behavior of `pv.PolyData.n_faces` has been deprecated.
                Use `pv.PolyData.n_cells` or `pv.PolyData.n_faces_strict` instead.
                See the documentation in '`pv.PolyData.n_faces` for more information.
  warnings.warn(


Widget(value='<iframe src="http://localhost:37597/index.html?ui=P_0x7fc561d47490_0&reconnect=auto" class="pyvi…

None

#### Save intersection info first

In [5]:
intersections = pymesh.detect_self_intersection(mesh)
intersecting_vertices, intersecting_faces = track_self_intersecting_faces(mesh, intersections)

#### Get outer hull

In [6]:
outer_hull = pymesh.compute_outer_hull(mesh)

In [7]:
self_intersection_stats(outer_hull)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |    1576 |
+-----------------------------------+---------+
| Number of faces                   |    3148 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


#### Map intersecting region to modified mesh & Extract submesh

In [8]:
mapped_vertices = map_to_modified_mesh(mesh, outer_hull, intersecting_vertices)

In [9]:
submesh, face_mask = extract_self_intersecting_region_from_modified(outer_hull, mapped_vertices)

In [10]:
visualize(submesh, filename="Submesh")

Widget(value='<iframe src="http://localhost:37597/index.html?ui=P_0x7fc561d478b0_1&reconnect=auto" class="pyvi…

None

In [11]:
remaining_mesh = extract_remaining_mesh(outer_hull, face_mask)

In [12]:
visualize(remaining_mesh, filename="remaining_mesh")

Widget(value='<iframe src="http://localhost:37597/index.html?ui=P_0x7fc5001ca7d0_2&reconnect=auto" class="pyvi…

None

#### Repair (or Remesh) the submesh
- `pymesh.collapse_short_edges(mesh, abs_threshold, rel_threshold, preserve_feature)`<br>
Collapses all edges with length less than a user specified threshold.

- `pymesh.split_long_edges(mesh, max_edge_length)`<br>
Splits long edges into 2 or more shorter edges.

- `pymesh.remove_obtuse_triangles(mesh, max_angle, max_iterations)`<br>
Splits each obtuse triangle into 2 or more right or sharp triangles.

- `pymesh.remove_degenerated_triangles(mesh, num_iterations)` <br>
Degenerate triangles are triangles with collinear vertices (i.e., some vertices are lying in the same straight line). They have zero areas and their normals are undefined.

In [13]:
from numpy.linalg import norm

def method2(mesh, detail="low"):

    bbox_min, bbox_max = mesh.bbox
    diag_len = norm(bbox_max - bbox_min)
    if detail == "normal":
        target_len = diag_len * 5e-3
    elif detail == "high":
        target_len = diag_len * 2.5e-3
    elif detail == "low":
        target_len = diag_len * 2e-2
    print("Target resolution: {} mm".format(target_len))

    count = 0
    mesh, __ = pymesh.remove_degenerated_triangles(mesh, 100)
    mesh, __ = pymesh.split_long_edges(mesh, target_len)
    num_vertices = mesh.num_vertices
    while True:
        mesh, __ = pymesh.collapse_short_edges(mesh, 1e-6, preserve_feature=True) # Remove extremely small edges
        mesh, __ = pymesh.collapse_short_edges(mesh, target_len,
                                               preserve_feature=True)
        mesh, __ = pymesh.remove_obtuse_triangles(mesh, 150.0, 100)
        if mesh.num_vertices == num_vertices:
            break

        num_vertices = mesh.num_vertices
        print("#v: {}".format(num_vertices))
        count += 1
        if count > 10: break

    mesh = pymesh.resolve_self_intersection(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh = pymesh.compute_outer_hull(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh, __ = pymesh.remove_obtuse_triangles(mesh, 179.0, 5)
    mesh, __ = pymesh.remove_isolated_vertices(mesh)

    return mesh

In [26]:
components = pymesh.separate_mesh(submesh)

In [27]:
repaired_components = []
for compoenent in components:
    repaired_component = method2(compoenent)
    repaired_components.append(repaired_component)

Target resolution: 0.06117549229246955 mm
#v: 511
#v: 505
Target resolution: 0.06117549229246955 mm
#v: 505
#v: 501


In [28]:
merged_components = pymesh.merge_meshes(repaired_components)

In [29]:
visualize(merged_components, filename="Repaired_submesh")

Widget(value='<iframe src="http://localhost:37597/index.html?ui=P_0x7fc4c0f1b280_9&reconnect=auto" class="pyvi…

None

#### Align submesh
Since we have modified the submesh, we need to change its boundary to what it used to be. Otherwise, when we combine the submesh and the remaining component later, there will be some small gaps.

In [30]:
aligned_submesh = align_submesh_boundary(remaining_mesh, merged_components)

In [31]:
visualize(aligned_submesh, filename="Aligned submesh")

Widget(value='<iframe src="http://localhost:37597/index.html?ui=P_0x7fc4c1a5e9b0_10&reconnect=auto" class="pyv…

None

#### Merge two meshes

In [32]:
repaired_full = replace_submesh_in_original(remaining_mesh, aligned_submesh)

In [33]:
visualize(repaired_full)

Widget(value='<iframe src="http://localhost:37597/index.html?ui=P_0x7fc4228bdba0_11&reconnect=auto" class="pyv…

None

#### Final refinement

In [34]:
final, _ = pymesh.remove_duplicated_vertices(repaired_full)
final, _ = pymesh.remove_duplicated_faces(final)
final, _ = pymesh.remove_degenerated_triangles(final)
final = pymesh.compute_outer_hull(final)
final, __ = pymesh.collapse_short_edges(final, 1e-6)
final, __ = pymesh.remove_obtuse_triangles(final, 175.0, 10)
final, _ = pymesh.remove_isolated_vertices(final)
final = pymesh.resolve_self_intersection(final)

In [35]:
self_intersection_stats(final)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |    1804 |
+-----------------------------------+---------+
| Number of faces                   |    3604 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


In [36]:
visualize(final, filename="Final Outerhull")

Widget(value='<iframe src="http://localhost:37597/index.html?ui=P_0x7fc4c1a5cfa0_12&reconnect=auto" class="pyv…

None

In [37]:
evaluation(outer_hull, final)

+-----------------------------------+-----------+-----------+
| Metric                            |    Before |     After |
+===================================+===========+===========+
| Number of vertices                | 1576      | 1804      |
+-----------------------------------+-----------+-----------+
| Number of faces                   | 3148      | 3604      |
+-----------------------------------+-----------+-----------+
| Number of intersecting face pairs |    0      |    0      |
+-----------------------------------+-----------+-----------+
| Volume                            |   11.0221 |   11.0157 |
+-----------------------------------+-----------+-----------+
| Area                              |   29.3751 |   29.3762 |
+-----------------------------------+-----------+-----------+


In [38]:
evaluate_displacement(outer_hull, final)

Max Vertex Displacement: 0.12689949559159355
Mean Vertex Displacement: 0.014576917342716766


In [39]:
pymesh.save_mesh("temp.ply", final, ascii=True)